In [0]:
-- This SQL code demonstrates how to create a tool that retrieves the contents of a web page
-- using a POST operation with the http_request function in Databricks.
-- For more details, refer to the Databricks documentation: 
-- https://docs.databricks.com/aws/en/sql/language-manual/functions/http_request

# Create the Connection
See https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-connection


In [0]:
DROP  CONNECTION IF EXISTS club_royale;

In [0]:
-- Set up a connect to the Club Royale Offers website
CREATE CONNECTION IF NOT EXISTS club_royale
  TYPE HTTP
  OPTIONS (
    host 'https://www.clubroyaleoffers.com',
    port '443',
    base_path '/',
    bearer_token ''
  )
  COMMENT 'Connection to Club Royale Offers API for player information retrieval.';

# Create the Connection Using Function

In [0]:
-- Function: GetClubRoyalePlayerInfo
-- Purpose: Retrieve player information from Club Royale using HTTP POST request.
-- Arguments:
--   player_id (STRING): The player's unique identifier.
--   last_name (STRING): The player's last name.
-- Returns:
--   STRING: The response from the Club Royale PlayerLookup API.

CREATE OR REPLACE FUNCTION clubroyale.offers.GetClubRoyalePlayerInfo(player_id STRING, last_name STRING)
RETURNS STRING
COMMENT 'Retrieves player information from Club Royale PlayerLookup API using player_id and last_name.'
RETURN (
  http_request(
    conn => 'club_royale',
    method => 'POST',
    path => '/PlayerLookup.asp',
    json => to_json(
      named_struct(
        'tbxLNameLookup', last_name,
        'tbxPlayerLookup', player_id
      )
    )
  ).text
);

In [0]:
-- Function: GetClubRoyalePlayerInfoDirect
-- Purpose: Retrieve player information from Club Royale using Requests object with a HTTP POST.
-- Arguments:
--   player_id (STRING): The player's unique identifier.
--   last_name (STRING): The player's last name.
-- Returns:
--   STRING: The response from the Club Royale PlayerLookup API.
CREATE OR REPLACE FUNCTION clubroyale.offers.GetClubRoyalePlayerInfoDirect(player_id STRING, last_name STRING)
RETURNS STRING
COMMENT 'Retrieves player information from Club Royale PlayerLookup API using player_id and last_name.'
LANGUAGE PYTHON
AS $$
import requests
url = f"https://www.clubroyaleoffers.com/PlayerLookup.asp"
payload = {
    "tbxLNameLookup": last_name,
    "tbxPlayerLookup": player_id
}
response = requests.post(url, data=payload)
print(response)
print(response.text)
return response.text
$$;